In [2]:
from google.colab import files
from google.colab import drive

import numpy as np
import pandas as pd

import nltk
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
nltk.download('punkt')

from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.initializers import Constant
from keras.layers import (LSTM, 
                          Embedding, 
                          BatchNormalization,
                          Dense, 
                          TimeDistributed, 
                          Dropout, 
                          Bidirectional,
                          Flatten, 
                          GlobalMaxPool1D)

from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

import pandas as pd
import numpy as np

from sklearn.metrics import (
    precision_score, 
    recall_score, 
    f1_score, 
    classification_report,
    accuracy_score
)

import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
drive.mount('/content/drive')
filename_CN = "/content/drive/My Drive/550FinalProject/CN_SampleData.csv"
glove_CN = "/content/drive/MyDrive/550FinalProject/glove.vectors.CN.txt"

Mounted at /content/drive


In [4]:
def read_file(filepath):
    data=pd.read_csv(filepath)
    return data
df = read_file(filename_CN)

In [5]:
data = df

In [6]:
data

,news,label
0,国家男篮孙悦深夜幽会富家女(组图)\n 荆楚网消息 (楚天金报) 去年国家男篮成绩骄人，“...,2
1,两市低开震荡 早盘终收红\n\n 本网讯(上海证券报 黄波)\n 12日早盘，沪深两市大...,13
2,前谷歌中国高管刘骏担任人民搜索首席科学家\n 人民网北京2月20日电 (记者 崔东) 近日...,12
3,加息预期渐近 外资行抢占利率互换市场\n “我们现在正和一些有中长期贷款的客户，或者负债率...,14
4,邓华德强硬回击质疑：电视解说员根本就不懂篮球！\n 本报安卡拉专电(记者潘天舒)中国队出线...,1
...,...,...
127595,明日股市三大猜想及应对策略\n 中财网\n 猜想一：\n 短线面临方向性选择？\n ...,13
127596,组图：柳敏最新个人写真集 泰国取景性感妩媚\n 新浪娱乐讯 北京时间2月5日下午消息，据...,2
127597,德国安联保险拟收购英吉利海峡隧道铁路\n 新浪财经讯 北京时间10月18日上午消息，据外电...,13
127598,高考生饮食别搞特殊 睡眠并不是越多越好\n 高考期间，考生和家长的紧张情绪会交融，家长会为...,6


In [7]:
def remove_pun_get_token(text: str) -> list:
    """
    Remove punctuation and tokenized strings

    Params:
        text {str} -- string of text

    Return:
        {list} -- tokenized string without punctuation

    """
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(text)

In [8]:
def drop_stopwords(contents, stopwords):
    contents_clean = []
    for line in contents:
        for word in line:
            if word in stopwords:
                continue
            contents_clean.append(word)
        #contents_clean.append(line_clean)
    return contents_clean

In [ ]:
data

,news,label
0,国家男篮孙悦深夜幽会富家女(组图)\n 荆楚网消息 (楚天金报) 去年国家男篮成绩骄人，“...,2
1,两市低开震荡 早盘终收红\n\n 本网讯(上海证券报 黄波)\n 12日早盘，沪深两市大...,13
2,前谷歌中国高管刘骏担任人民搜索首席科学家\n 人民网北京2月20日电 (记者 崔东) 近日...,12
3,加息预期渐近 外资行抢占利率互换市场\n “我们现在正和一些有中长期贷款的客户，或者负债率...,14
4,邓华德强硬回击质疑：电视解说员根本就不懂篮球！\n 本报安卡拉专电(记者潘天舒)中国队出线...,1
...,...,...
127595,明日股市三大猜想及应对策略\n 中财网\n 猜想一：\n 短线面临方向性选择？\n ...,13
127596,组图：柳敏最新个人写真集 泰国取景性感妩媚\n 新浪娱乐讯 北京时间2月5日下午消息，据...,2
127597,德国安联保险拟收购英吉利海峡隧道铁路\n 新浪财经讯 北京时间10月18日上午消息，据外电...,13
127598,高考生饮食别搞特殊 睡眠并不是越多越好\n 高考期间，考生和家长的紧张情绪会交融，家长会为...,6


In [ ]:
def preprocess_CN(lst: list, stopwords: bool = False):
    stopword = ['的','呀','这','那','就','的话','如果']
    preprocess = []
    for sent in lst:
        
        seq = remove_pun_get_token(sent)

        if stopwords == True:
            seq = drop_stopwords(seq,stopword)

        preprocess.append(" ".join(seq[:200]))
        
    return preprocess

In [ ]:
X=data["news"]
y = data['label']

In [ ]:
X1 = preprocess_CN(X.values,True)

In [ ]:
len(X1[1])

399

In [ ]:
def split_dataset(X,y, test_size = 0.2):
    """
    Prepare the training/testing dataset

    Params:

        test_size {float} -- precentage of testing dataset
    Return:
        X_train,X_test,y_train,y_test {tuple} -- split datasets

    """
    
    return train_test_split(X, y, test_size=test_size, random_state=42)

In [ ]:
X = pd.DataFrame(X1)
X = X[0]
X

0         国 家 男 篮 孙 悦 深 夜 幽 会 富 家 女 组 图 荆 楚 网 消 息 楚 天 金 ...
1         两 市 低 开 震 荡 早 盘 终 收 红 本 网 讯 上 海 证 券 报 黄 波 1 2 ...
2         前 谷 歌 中 国 高 管 刘 骏 担 任 人 民 搜 索 首 席 科 学 家 人 民 网 ...
3         加 息 预 期 渐 近 外 资 行 抢 占 利 率 互 换 市 场 我 们 现 在 正 和 ...
4         邓 华 德 强 硬 回 击 质 疑 电 视 解 说 员 根 本 不 懂 篮 球 本 报 安 ...
                                ...                        
127595    明 日 股 市 三 大 猜 想 及 应 对 策 略 中 财 网 猜 想 一 短 线 面 临 ...
127596    组 图 柳 敏 最 新 个 人 写 真 集 泰 国 取 景 性 感 妩 媚 新 浪 娱 乐 ...
127597    德 国 安 联 保 险 拟 收 购 英 吉 利 海 峡 隧 道 铁 路 新 浪 财 经 讯 ...
127598    高 考 生 饮 食 别 搞 特 殊 睡 眠 并 不 是 越 多 越 好 高 考 期 间 考 ...
127599    中 国 建 筑 国 际 前 9 月 经 营 溢 利 7 亿 港 元 中 国 建 筑 附 属 ...
Name: 0, Length: 127600, dtype: object

In [ ]:
def embed(corpus): 
    return word_tokenizer.texts_to_sequences(corpus)

In [ ]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype="uint8")[y]

In [ ]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(X)
vocab_length = len(word_tokenizer.word_index) + 1
longest_train = max(X, key=lambda sentence: len(word_tokenize(sentence)))
length_long_sentence = len(word_tokenize(longest_train))
padded_sentences = pad_sequences(embed(X), length_long_sentence, padding='post')

In [ ]:
embeddings_dictionary = {}
embedding_dim = 50
glove_file = open(glove_CN, encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions

glove_file.close()

In [ ]:
def embedding_mat():
  
  word_tokenizer = Tokenizer()
  word_tokenizer.fit_on_texts(X)

  embedding_matrix = np.zeros((vocab_length, embedding_dim))
  for word, index in word_tokenizer.word_index.items():
      if index >= vocab_length:
          continue
      embedding_vector = embeddings_dictionary.get(word)
      if embedding_vector is not None:
          embedding_matrix[index] = embedding_vector
  return embedding_matrix

In [ ]:
embedding_matrix = embedding_mat()

In [ ]:
def BLSTM():
    
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0], 
                        output_dim=embedding_matrix.shape[1], 
                        weights = [embedding_matrix], 
                        input_length=length_long_sentence,
                        trainable=False))
    
    model.add(Bidirectional(LSTM(length_long_sentence, return_sequences = True, recurrent_dropout=0.2)))
    model.add(GlobalMaxPool1D())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(length_long_sentence, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(length_long_sentence, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(14, activation = 'sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
y = to_categorical(y, 15)
y= y[:,1:]


In [ ]:
X_train, X_test, y_train, y_test = split_dataset(padded_sentences,y,0.2)

In [ ]:
if __name__ == "__main__":
  from keras.utils.layer_utils import count_params

  print("THUCNews [Training: 80%, Testing： 20%]")
  model = BLSTM()
  print(model.summary())
  print("trainable:", count_params(model.trainable_weights))
  print("non trainable:", count_params(model.non_trainable_weights))
  model.fit(X_train,
              y_train,
              batch_size=32,
              epochs=2,
              validation_data=[X_test, y_test],
              verbose=1)

THUCNews [Training: 80%, Testing： 20%]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 50)           288550    
                                                                 
 bidirectional (Bidirectiona  (None, 200, 400)         401600    
 l)                                                              
                                                                 
 global_max_pooling1d (Globa  (None, 400)              0         
 lMaxPooling1D)                                                  
                                                                 
 batch_normalization (BatchN  (None, 400)              1600      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 400)               0         
                 

KeyboardInterrupt: ignored